# Demo Notebook: Flood Detection with Sen1Floods11 Dataset on SharingHub

## Objective

This notebook demonstrates how to organize, train, and use a deep learning model for image segmentation through the flood detection using the [**Sen1Floods11 dataset**](https://github.com/cloudtostreet/Sen1Floods11?tab=readme-ov-file), leveraging the [**SharingHub**](https://sharinghub.p2.csgroup.space/#/) platform.

### Key Features of this Demo:
1. **Dataset Management**: Retrieve and version datasets stored on SharingHub, using DVC and GitLab integration.
2. **Experiment Tracking**: Configure and use MLflow for tracking training experiments, directly linked to the GitLab repository.
3. **Model Training**: Train the flood detection model with data streamed from SharingHub’s STAC API or its DVC remote.
4. **Model Inference**: Use the trained model to perform segmentation on unseen data, predicting flooded areas.
5. **Automation**: Streamline operations with Docker containers and CWL workflows for reproducibility.

---

## Workflow Overview

### 1. **Training the Model**
- Store the model in onnx format.
- Get metrics to evaluate the model performances.
- Track the training progress using MLflow, with runs linked to the model’s GitLab repository.

### 2. **Inference**
- Perform segmentation on Sentinel-1 images to detect flooded areas.
- Visualize results to evaluate model performance.

### 3. **Reproducibility**
- Use Docker containers and CWL workflows for consistent environment setup and execution.

---

# prerequisites

Before entering the demonstration, make sure you have followed the tutorial in the [readme](../README.md) to be sure you have correctly configured your environment as well as your environment variables

# 1. **Training the model**

In [1]:
!poetry run python3 src/train.py

STREAMING MODE DISABLED.
sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/S1Hand/Ghana_103272_S1Hand.tif sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/LabelHand/Ghana_103272_LabelHand.tif
sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/S1Hand/USA_831672_S1Hand.tif sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/LabelHand/USA_831672_LabelHand.tif
sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/S1Hand/Pakistan_246510_S1Hand.tif sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/LabelHand/Pakistan_246510_LabelHand.tif
sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/S1Hand/Ghana_5079_S1Hand.tif sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/LabelHand/Ghana_5079_LabelHand.tif
Training Loop: 100%|████████████████████████████| 16/16 [01:42<00:00,  6.39s/it]
Current Epoch: 0
model saved at checkpoints/Sen1Floods11_0_0.5324235558509827.onnx .
model saved in MLflow.
Training Loss: tensor(0.3631, grad_fn=<DivBackward0>)
Training IOU: 

# 2. **Inference**

In [4]:
!poetry run python3 src/inference.py checkpoints/Sen1Floods11_0_0.5194225907325745.onnx sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/S1Hand/India_80221_S1Hand.tif

image saved at predictions/prediction.tif


# 3. **Reproducibility**
## **Docker**

### Create docker image for training from Dockerfile.train

Add your credentials in a .env file and run the docker image:
-   MLFLOW_TRACKING_TOKEN=
-   LOGNAME=
-    ACCESS_KEY_ID= 
-    SECRET_ACCESS_KEY= 


In [9]:
!docker build -f Dockerfile.train -t train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 572B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 572B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.3s
[+] Building 0.4s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 572B                                       0.0s
 => [internal] load metadata for docker.

### Streaming Mode in `train`

The *streaming* mode in `train` allows data to be fetched on-the-fly, image by image, during training. This eliminates the need to download the entire dataset, saving both time and storage space. Data is progressively loaded from remote storage via DVC, reducing memory usage and optimizing the training process. This mode is ideal for handling large datasets without overwhelming local resources.


In [10]:
!docker run -it --env-file .env train bash

STREAMING MODE ENABLED.
CACHE IS ENABLED.
Training Loop: 16it [04:49, 18.12s/it]
Training Loop: 16it [01:15,  4.72s/it]
Training Loop: 16it [01:16,  4.77s/it]
Training Loop: 16it [01:16,  4.81s/it]
Training Loop: 16it [01:19,  4.95s/it]
Training Loop: 16it [01:30,  5.68s/it]
Training Loop: 16it [01:33,  5.86s/it]
Training Loop: 16it [01:33,  5.86s/it]
Training Loop: 16it [01:25,  5.32s/it]
Training Loop: 16it [01:24,  5.28s/it]
Current Epoch: 0
model saved at checkpoints/Sen1Floods11_0_0.17997555434703827.onnx .
model saved in MLflow.
Training Loss: tensor(0.3634, grad_fn=<DivBackward0>)
Training IOU: tensor(0.3858)
Training Accuracy: tensor(0.8839)
Validation Loss: tensor(0.6157)
Validation IOU: tensor(0.1800)
Validation Accuracy: tensor(0.9158)
max valid iou: tensor(0.1800)
🏃 View run unequaled-finch-393 at: https://sharinghub.p2.csgroup.space/mlflow/space_applications/mlops-services/sample-projects/ai-models/flood-model/tracking/#/experiments/23/runs/75607d930ef8470daa319392c4d55c28

### Create docker image for inference from Dockerfile.inf

In [ ]:
!docker build -f Dockerfile.inf -t inference .

### Run the docker image for inference

In [ ]:
!docker run inference checkpoints/Sen1Floods11_1_0.4923180341720581.onnx sen1floods11-dataset/v1.1/data/flood_events/HandLabeled/S1Hand/India_80221_S1Hand.tif && ls predictions/

## **CWL**

### Run the docker image using cwl with custom parameters saved in run_inference_input.yml

In [ ]:
!cwltool --debug run_inference.cwl run_inference_input.yml 